# Time dependent D3D simulation

In [ ]:
using Plots;
using FUSE
using Interact
FUSE.ProgressMeter.ijulia_behavior(:clear);

In [ ]:
use_local_cache = true
@time begin
#shot = 133221; ini, act = FUSE.case_parameters(:D3D, shot; fit_profiles=true, use_local_cache); # ECH /// measured 6*C is higher than ne
#shot = 200000; ini, act = FUSE.case_parameters(:D3D, shot; fit_profiles=true, use_local_cache); # ECH
#ini, act = FUSE.case_parameters(:D3D, 170325); # ECH 1.67924
shot = 168830; ini, act = FUSE.case_parameters(:D3D, shot; fit_profiles=true, use_local_cache); #NBI with balanced torque

#shot = 156905; ini, act = FUSE.case_parameters(:D3D, shot; use_local_cache); # Weird ZIPFIT holds <--fast_particles_profiles!
#shot = 194306; ini, act = FUSE.case_parameters(:D3D, shot; use_local_cache, new_impurity_match_power_rad=:Kr); # negT
#ini, act = FUSE.case_parameters(:D3D, 177476; use_local_cache, PROFILES_tree="ZIPFIT02"); # Cihan's
#@time ini, act = FUSE.case_parameters(:D3D, 168830);
#ini, act = FUSE.case_parameters(:D3D, 200204); # beam power scan, rotation, lower_to_upper null
#ini, act = FUSE.case_parameters(:D3D, 200021; EFIT_tree="EFIT01"); # negative B
#ini, act = FUSE.case_parameters(:D3D, 173389);
end
@checkin :fetch ini act

In [ ]:
@checkout :fetch ini act;
ini.time.simulation_start = ini.general.dd.equilibrium.time_slice[2].time
dd = IMAS.dd()
@time FUSE.init!(dd, ini, act);
@checkin :init dd act;

In [ ]:
@checkout :init dd act;
@manipulate for time0 in slider(ini.general.dd.equilibrium.time, label="time")
    dd.global_time = time0

    # plot(dd.thomson_scattering, :n_e, label="")
    # plot!(dd.charge_exchange, :n_imp, normalization=6.0, label="")
    # plot!(dd.core_profiles.profiles_1d[].electrons, :density_thermal, lw=2, label="ne")
    # plot!(dd.core_profiles.profiles_1d[].ion[1], :density_thermal, normalization=1, lw=2, label="nD")
    # plot!(dd.core_profiles.profiles_1d[].ion[2], :density_thermal, normalization=6, lw=2, label="nC")
    # plot!(xlim=(0,1.2), ylim=(0,1.1E20))
    
    # plot(dd.charge_exchange, :t_i, label="")
    # plot!(dd.core_profiles.profiles_1d[], :t_i_average, lw=2, label="Ti")
    # plot!(xlim=(0,1.2), ylim=(0,5000))
    
    # plot(dd.charge_exchange, :zeff, label="")
    # plot!(dd.core_profiles.profiles_1d[], :zeff, lw=2, label="Zeff")
    # plot!(xlim=(0,1.2), ylim=(0.0,6))
        
    plot(dd.thomson_scattering, :t_e, label="")
    plot!(dd.core_profiles.profiles_1d[].electrons, :temperature, lw=2, label="Te")
    plot!(xlim=(0,1.2), ylim=(0,5000))
    
end

In [ ]:
@checkout :init dd act;
experiment_LH = FUSE.LH_analysis(dd)#; scale_LH=2.5);
;

In [ ]:
@checkout :init dd act;
act.ActorPedestal.model = :dynamic
act.ActorPedestal.tau_n = experiment_LH.tau_n
act.ActorPedestal.tau_t = experiment_LH.tau_t
act.ActorWPED.ped_to_core_fraction = experiment_LH.W_ped_to_core_fraction
act.ActorEPED.ped_factor = 1.0
act.ActorPedestal.T_ratio_pedestal = 1.0 # Ti/Te in the pedestal
if true
    # density and Zeff from experiment
    act.ActorPedestal.density_ratio_L_over_H = 1.0
    act.ActorPedestal.zeff_ratio_L_over_H = 1.0
else
    # density can go from L to H mode at a different time
    act.ActorPedestal.density_ratio_L_over_H = experiment_LH.ne_L_over_H
    act.ActorPedestal.zeff_ratio_L_over_H = experiment_LH.zeff_L_over_H
    dd.pulse_schedule.density_control.n_e_line.reference = experiment_LH.ne_H
    dd.pulse_schedule.density_control.zeff_pedestal.reference = experiment_LH.zeff_H
end

if false
    # LH-transition from LH scaling law
    act.ActorPedestal.mode_transitions = missing
else
    # LH-transition at user-defined times
    act.ActorPedestal.mode_transitions = experiment_LH.mode_transitions
    act.ActorPedestal.mode_transitions[5.2] = :L_mode
    @show act.ActorPedestal.mode_transitions
end

act.ActorEquilibrium.model = :EGGO # EGGO

act.ActorFRESCO.nR = 65
act.ActorFRESCO.nZ = 65

act.ActorNeutralFueling.τp_over_τe = 0.5

act.ActorFluxMatcher.evolve_plasma_sources = false
act.ActorFluxMatcher.algorithm = :simple
act.ActorFluxMatcher.max_iterations = -10
act.ActorFluxMatcher.verbose = false
act.ActorFluxMatcher.evolve_pedestal = false
#act.ActorFluxMatcher.evolve_densities = :zeff

act.ActorTGLF.tglfnn_model = "sat1_em_d3d"
#act.ActorTGLF.model = :TJLF
# act.ActorTGLF.tglfnn_model = "sat3_em_d3d_azf-1"

dd.global_time = ini.general.dd.equilibrium.time_slice[2].time # start_time
δt = 0.025
final_time = ini.general.dd.equilibrium.time[end]
act.ActorDynamicPlasma.Nt = Int(ceil((final_time - dd.global_time) / δt))
act.ActorDynamicPlasma.Δt = final_time - dd.global_time

act.ActorDynamicPlasma.evolve_current = true
act.ActorDynamicPlasma.evolve_equilibrium = true
act.ActorDynamicPlasma.evolve_transport = true
act.ActorDynamicPlasma.evolve_hcd = true
act.ActorDynamicPlasma.evolve_pf_active = false
act.ActorDynamicPlasma.evolve_pedestal = true

act.ActorDynamicPlasma.ip_controller = false
act.ActorDynamicPlasma.time_derivatives_sources = true

# act.ActorCurrent.model = :replay
# act.ActorEquilibrium.model = :replay
# act.ActorCoreTransport.model = :replay
# act.ActorPedestal.model = :replay
# act.ActorHCD.ec_model = :replay
# act.ActorHCD.ic_model = :replay
# act.ActorHCD.lh_model = :replay
# act.ActorHCD.nb_model = :replay
# act.ActorHCD.pellet_model = :replay
# act.ActorHCD.neutral_model = :none
GC.gc()
@time actor = FUSE.ActorDynamicPlasma(dd, act; verbose=true);
@checkin :time_dep_allst dd ini act actor;

In [ ]:
# IMAS.imas2hdf(dd, "fuse_$(shot).h5");

In [ ]:
#@checkout :time_dep dd;

dd1 = FUSE.checkpoint[:init].dd
IMAS.trim_time!(dd, (-Inf, dd.global_time-1E-6)); # shaves off the last time slice, useful when simulation crashes and leaves last time-slice in a bad state

@manipulate for time0 in slider(dd.equilibrium.time, value=dd.global_time/2.0, label="time")
    try
        FUSE.plot_plasma_overview(dd, Float64(time0); dd1, aggregate_hcd=true)#, min_power=1E4)
        # plot(dd.core_profiles.profiles_1d[time0], :zeff; ylim=(0,6))

    IMAS.ylim(Dict{Int,Float64}(
        -2 => 0.0, 
        -3 => 0.0, 3 => 1500.0,
        4 => 4.0,
        5 => 4.0,
        6 => 1.1E20,

        -8 => -0.2, 8 => 2.0,
        -9 => -0.25, 9 => .5,
        -10 => -0.25, 10 => .5,
        -11 => -1E20, 11 => 1.0E20,
        
        -14 => 0.0, 14 => 0.101,
        -15 => 0.0, 15 => 0.101,
        -16 => -2.0E19, 16 => 2.0E19))

#         p=plot(dd1.equilibrium; time0, color=:black)
#         plot!(dd.equilibrium; time0)
#         plot!(p[1], dd.wall)


#         plot(dd.core_transport; time0)

#         plot(dd.core_sources; time0, aggregate_radiation=true, aggregate_hcd=true)

    catch
        plot()
    end
    
end

In [ ]:
using Interact
using Printf

a = @animate for (k, time0) in enumerate(dd.equilibrium.time)
    println(k)
    try
        FUSE.plot_plasma_overview(dd, time0; dd1=dd1, aggregate_hcd=true)
    catch
        continue
    end
    
    IMAS.ylim(Dict{Int,Float64}(
        -2 => 0.0, 
        -3 => 0.0, 3 => 1500.0,
        4 => 4.0,
        5 => 4.0,
        6 => 1.1E20,

        -8 => -0.2, 8 => 2.0,
        -9 => -0.25, 9 => .5,
        -10 => -0.25, 10 => .5,
        -11 => -1E20, 11 => 1.0E20,
        
        -14 => 0.0, 14 => 0.101,
        -15 => 0.0, 15 => 0.101,
        -16 => -2.0E19, 16 => 2.0E19))

    savefig("D3D_$(shot)/D3D_$(shot)___$(@sprintf("%04d", k)).png") # magick -delay 2 -loop 0 D3D_168830___*.png -layers Optimize D3D_194306.gif
end
gif(a, "D3D_$(shot)/D3D_$(shot).gif", fps=12)

In [ ]:
FUSE.group_similar_actuators(dd.nbi.unit, dd.pulse_schedule.nbi.unit; grouping_tolerance=1.0)